1. Import Dependencies

In [ ]:
import cv2
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import mediapipe as mp

In [ ]:
mp_holistic = mp.solutions.holistic  
mp_drawing = mp.solutions.drawing_utils 

In [ ]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

    return image,results

In [ ]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [ ]:
def draw_styled_landmarks(image,results):
    
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS
                              ,mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4)
                              ,mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2))
    
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS
                              ,mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4)
                              ,mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))

2. KeyPoints using MP Holistic

In [ ]:
capt = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistics:

    while True:

        ret,frame = capt.read()

        image, results = mediapipe_detection(frame,holistics)

        draw_styled_landmarks(image,results)

        cv2.imshow('capture',image)

        if cv2.waitKey(10) & 0xFF == ord('a'):
            break

    capt.release()
    cv2.destroyAllWindows()


In [ ]:
len(results.right_hand_landmarks.landmark)
len(results.left_hand_landmarks.landmark)

In [ ]:
image

In [ ]:
draw_landmarks(image,results)

In [ ]:
plt.imshow(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))

3. EXTRACT KEYPOINT VALUES

In [ ]:
results.pose_landmarks

In [ ]:
print(len(results.right_hand_landmarks.landmark))
print(len(results.left_hand_landmarks.landmark))

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3) 
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([pose,face,lh,rh])

In [ ]:
extract_keypoints(results).shape

In [ ]:
33*4+468*3+21*3+21*3

4. Setup Folders For Collections

In [ ]:
DATA_PATH = os.path.join('D:\\MP_Data')
actions = np.array(['hello','iloveyou','good','okay'])
no_sequence = 30
sequence_length = 30 

In [ ]:
for action in actions:
    for sequence in range(no_sequence):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

5. Collect Keypoint Sequences

In [ ]:
capt = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistics:

    for action in actions:
        for sequence in range(no_sequence):
            for frame_num in range(sequence_length):


                ret,frame = capt.read()

                image, results = mediapipe_detection(frame,holistics)

                draw_styled_landmarks(image,results)

                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)

                    cv2.putText(image, 'Collecting frames for {} video number {}'.format(actions,sequence), (15,12)
                                , cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),4,cv2.LINE_AA)
                    
                    cv2.waitKey(2000)

                else:
                    cv2.putText(image, 'Collecting frames for {} video number {}'.format(action,sequence), (15,12)
                                , cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),4,cv2.LINE_AA)
                    
                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
                np.save(npy_path,keypoints)


                cv2.imshow('capture',image)

                if cv2.waitKey(10) & 0xFF == 27:
                    break

    capt.release()
    cv2.destroyAllWindows()


In [ ]:
capt.release()
cv2.destroyAllWindows()

6. Preprocess Data and Create Labels and Features

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num,label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequence):
        window=[]
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence), '{}.npy'.format(frame_num)))
            window.append(res)

        sequences.append(window)
        labels.append(label_map[action])


In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [ ]:
X= np.array(sequences)
y = to_categorical(labels,4).astype(int)

In [ ]:
y


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.05,random_state=42)

7. BUILD AND TRAIN LSTM NEURAL NETWORK

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('D:\\Logs')
tb_callbacks = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=500, verbose =2)

8. Make Predictions

In [ ]:
pred_prob = model.predict(X_test)

In [ ]:
pred_prob

In [ ]:
np.sum(pred_prob[0])

In [ ]:
np.argmax(pred_prob[0]) # reyurn index number

In [ ]:
actions[np.argmax(pred_prob[4])]

In [ ]:
actions[np.argmax(y_test[2])]

9. Save weights

In [ ]:
model.save("D:\\MP_DATA\\actions.h5")

In [ ]:
model.load_weights("D:\\MP_DATA\\actions.h5")

10. Evaluation Using Confusion Metric and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
y_true = np.argmax(y_test,axis=1).tolist()
y_pred = np.argmax(pred_prob,axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(y_true,y_pred)

In [ ]:
accuracy_score(y_true,y_pred)

11.     Test In Real Time

In [ ]:
from scipy import stats

In [ ]:
def prob_viz(res, actions, input_frame):
    output_frame = input_frame.copy()
    
    for num, (action, prob) in enumerate(zip(actions, res)):
        # Display the action name and its probability
        text = f"{action}: {prob:.2f}"
        cv2.putText(output_frame, text, (40, 60 + num * 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    return output_frame

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('D:\\output3.avi', fourcc, 6.0, (640, 480))

sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        
        ret, frame = cap.read()

        if not ret:
            break

       
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        
        draw_styled_landmarks(image, results)
        
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
            
           
            if np.unique(predictions[-10:])[0] == np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 1: 
                sentence = sentence[-1:]

            
            image = prob_viz(res, actions, image)
            
        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        out.write(image)
        
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

out.release()
cap.release()
cv2.destroyAllWindows()


In [ ]:
np.max(res)

In [ ]:
cap.release()
cv2.destroyAllWindows()